In [42]:
import pandas as pd 
import numpy as np 
import sklearn

In [43]:
train = pd.read_csv("/Users/kevinfinn/Downloads/titanic/train.csv")
test = pd.read_csv("/Users/kevinfinn/Downloads/titanic/test.csv")

In [44]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [45]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
y = enc.fit_transform(train.Survived)

In [46]:
X = train[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]

In [47]:
X.isnull().sum()

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [48]:
X.Pclass.value_counts()

Pclass
3    491
1    216
2    184
Name: count, dtype: int64

In [21]:
X.Sex.value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

In [23]:
X.SibSp.value_counts()

SibSp
0    608
1    209
2     28
4     18
3     16
8      7
5      5
Name: count, dtype: int64

In [24]:
X.Parch.value_counts()

Parch
0    678
1    118
2     80
5      5
3      5
4      4
6      1
Name: count, dtype: int64

In [27]:
X.Cabin.value_counts()

Cabin
B96 B98        4
G6             4
C23 C25 C27    4
C22 C26        3
F33            3
              ..
E34            1
C7             1
C54            1
E36            1
C148           1
Name: count, Length: 147, dtype: int64

In [31]:
X.Embarked.value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [32]:
X.Parch.value_counts()

Parch
0    678
1    118
2     80
5      5
3      5
4      4
6      1
Name: count, dtype: int64

In [49]:
categorical_columns = ['Pclass', 'Sex', 'Embarked', 'SibSp', 'Parch', 'Embarked']
for column in categorical_columns:
    X[column] = enc.fit_transform(X[column].astype(str))

/var/folders/l2/yg146pls5_q015mz26q0pyfm0000gn/T/ipykernel_18288/612272645.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = enc.fit_transform(X[column].astype(str))
/var/folders/l2/yg146pls5_q015mz26q0pyfm0000gn/T/ipykernel_18288/612272645.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = enc.fit_transform(X[column].astype(str))
/var/folders/l2/yg146pls5_q015mz26q0pyfm0000gn/T/ipykernel_18288/612272645.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [50]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [52]:
model = XGBClassifier()
print(cross_val_score(model, X, y, cv=10).mean())

0.8148439450686642


In [56]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [58]:
params = {"max_depth": np.arange(3,10),
              "min_child_weight": np.arange(1,10),
              "subsample":np.arange(0.5,1,0.1),
              "colsample_bytree":np.arange(0.5,1,0.1),
              "colsample_bylevel":np.arange(0.5,1,0.1),
              "eta":np.arange(0.01,0.3,0.3),
              "gamma":np.arange(0,5),
              "n_estimators":np.arange(100,1000,100)}

grid_search = GridSearchCV(estimator=model, param_grid=params, cv=10, scoring='accuracy', verbose=1)

grid_search.fit(X, y)

print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score achieved: ", grid_search.best_score_)

Fitting 10 folds for each of 354375 candidates, totalling 3543750 fits
